In [1]:
import torch
from torch import nn


class TheHammer(nn.Module):
    def __init__(self, num_hidden=2, dropout=0., in_feats=2048, num_classes=2):
        super(TheHammer, self).__init__()
        
        self.extractor = nn.Sequential(*[
            nn.Sequential(
                nn.Linear(in_feats//(l+1), in_feats//(l+2)),
                nn.ReLU()
            )
            for l in range(num_hidden)
        ])
        
        self.classifier = nn.Sequential(
            nn.Dropout(dropout),
            nn.Linear(in_feats//(num_hidden+1), num_classes)
        )
        
    def forward(self, x):
        x = self.extractor(x)
        return self.classifier(x)

In [83]:
import torch
from torch import nn


class TheFuckingSledge(nn.Module):
    def __init__(self, num_hidden=2, dropout=0., in_feats=2048, num_classes=2):
        super(TheFuckingSledge, self).__init__()
        
        self.block1 = nn.Sequential(
            nn.Conv1d(1, 8, kernel_size=7, padding=3),
            nn.BatchNorm1d(8),
            nn.ReLU()
        )
        self.block2 = nn.Sequential(
            nn.Conv1d(8, 16, kernel_size=5, stride=2, padding=2),
            nn.BatchNorm1d(16),
            nn.ReLU()
        )
        self.block3 = nn.Sequential(
            nn.Conv1d(16, 16, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm1d(16),
            nn.ReLU()
        )
        self.avgpool = nn.AvgPool1d(2, stride=2)
        self.block4 = nn.Sequential(
            nn.Conv1d(16, 32, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm1d(32),
            nn.ReLU()
        )
        self.fc = nn.Linear(2048, 2)
        
    def forward(self, x):
        x = self.block1(x.unsqueeze(1))
        x = self.block2(x)
        x = self.block3(x)
        x = self.avgpool(x)
        x = self.block4(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        return self.fc(x)

In [84]:
TheFuckingSledge(dropout=0.5)

TheFuckingSledge(
  (block1): Sequential(
    (0): Conv1d(1, 8, kernel_size=(7,), stride=(1,), padding=(3,))
    (1): BatchNorm1d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (block2): Sequential(
    (0): Conv1d(8, 16, kernel_size=(5,), stride=(2,), padding=(2,))
    (1): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (block3): Sequential(
    (0): Conv1d(16, 16, kernel_size=(3,), stride=(2,), padding=(1,))
    (1): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (avgpool): AvgPool1d(kernel_size=(2,), stride=(2,), padding=(0,))
  (block4): Sequential(
    (0): Conv1d(16, 32, kernel_size=(3,), stride=(2,), padding=(1,))
    (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (fc): Linear(in_features=2048, out_features=2, bias=True)
)

In [85]:
import numpy as np

from torch.utils.data import Dataset
from torchvision.transforms import ToTensor

class TheHammerSack(Dataset):
    def __init__(self, bag_of_tools: np.array, is_a_hammer: np.array, transform=None, target_transform=None):
        super(TheHammerSack, self).__init__()
        
        self.bag_of_tools = bag_of_tools
        self.is_a_hammer = is_a_hammer
        self.transform = transform
        self.target_transform = target_transform
        
    def __len__(self):
        return len(self.bag_of_tools)
    
    def __getitem__(self, ix):
        tool = self.bag_of_tools[ix]
        is_this_a_hammer = self.is_a_hammer[ix]
        
        if self.transform is not None:
            tool = self.transform(tool)
        if self.target_transform is not None:
            is_this_a_hammer = self.target_transform(is_this_a_hammer)
            
        return tool, is_this_a_hammer

In [86]:
import os
import numpy as np
 
hammers_one = np.load(os.path.join('out', f'val-id_layer4.npy')).astype('float32')

hammers_two = np.load(os.path.join('out', f'test-id_layer4.npy')).astype('float32')
is_a_hammer_two = np.load(os.path.join('out', f'test-id_layer4_labels.npy')).astype('float32')
hammers_two = np.concatenate((hammers_two[0, is_a_hammer_two == 0], hammers_two[1, is_a_hammer_two == 1]), axis=0)

hammers = np.concatenate((hammers_one, hammers_two), axis=0)

other_tools = np.load(os.path.join('out', f'test-ood_layer4.npy'))[0].astype('float32')

In [87]:
from sklearn.model_selection import train_test_split

hammers_train, hammers_test = train_test_split(hammers, test_size=0.1)
hammers_train, hammers_val = train_test_split(hammers_train, test_size=0.1481)

other_tools_train, other_tools_test = train_test_split(other_tools, test_size=0.66)
other_tools_train, other_tools_val = train_test_split(other_tools_train, test_size=0.15)

print(f"data_distribution: \n"
    f"hammers: {len(hammers_train)} {len(hammers_val)} {len(hammers_test)},\n"
    f"other_tools: {len(other_tools_train)} {len(other_tools_val)} {len(other_tools_test)}")

data_distribution: 
hammers: 2300 400 300,
other_tools: 2312 408 5280


In [88]:
from torch.utils.data import DataLoader

training_tools = np.concatenate((hammers_train, other_tools_train), axis=0)
training_is_a_hammer = np.concatenate((np.ones(len(hammers_train)), np.zeros(len(other_tools_train))), axis=0).astype('int')

shuffle_index = np.arange(len(training_tools))
np.random.shuffle(shuffle_index)

training_hammersack = TheHammerSack(training_tools[shuffle_index], training_is_a_hammer[shuffle_index])
training_hammersack = DataLoader(training_hammersack, batch_size=64, pin_memory=True, drop_last=False, shuffle=True)

In [89]:
validation_tools = np.concatenate((hammers_val, other_tools_val), axis=0)
validation_is_a_hammer = np.concatenate((np.ones(len(hammers_val)), np.zeros(len(other_tools_val))), axis=0).astype('int')

shuffle_index = np.arange(len(validation_tools))
np.random.shuffle(shuffle_index)

validation_hammersack = TheHammerSack(validation_tools[shuffle_index], validation_is_a_hammer[shuffle_index])
validation_hammersack = DataLoader(validation_hammersack, batch_size=64, pin_memory=True, drop_last=False, shuffle=False)

In [90]:
from torch.nn import CrossEntropyLoss
from torch.optim import Adam, SGD
from torch.optim.lr_scheduler import ReduceLROnPlateau

device = 'cuda:0'
# the_hammer = TheHammer(dropout=0.5).to(device)
the_hammer = TheFuckingSledge(dropout=0.5).to(device)
criterion = CrossEntropyLoss(reduction='mean').to(device)
optimizer = SGD(params=the_hammer.parameters(), lr=0.01, weight_decay=5e-5)
scheduler = ReduceLROnPlateau(optimizer, patience=10, cooldown=10, factor=0.1, verbose=True)

In [91]:
import torch
from collections import defaultdict
from tqdm import tqdm
import json

In [92]:
from inferno.utils.train_utils import AverageMeter
from sklearn.metrics import classification_report

best_loss = np.inf
for epoch in range(100):
    print(f'epoch: {epoch}')
    
    the_hammer.train()
    train_trackers = defaultdict(AverageMeter)
    all_labels, all_preds = [], []
    for tools, is_it_a_hammer in training_hammersack:
        tools = tools.to(device)
        is_it_a_hammer = is_it_a_hammer.to(device)
        
        logits = the_hammer(tools)
        loss = criterion(logits, is_it_a_hammer)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        all_labels.extend(is_it_a_hammer.cpu().detach().numpy().tolist())
        all_preds.extend(logits.argmax(dim=1).detach().cpu().numpy().tolist())
        train_trackers['loss'].update(loss.item())
        
    print('train: {}'.format({k: v.avg for k, v in train_trackers.items()}))
    print(f'train acc: {classification_report(y_true=all_labels, y_pred=all_preds, output_dict=True)["accuracy"]}')
        
    the_hammer.eval()
    val_trackers = defaultdict(AverageMeter)
    all_labels, all_preds = [], []
    for tools, is_it_a_hammer in validation_hammersack:
        tools = tools.to(device)
        is_it_a_hammer = is_it_a_hammer.to(device)
        
        with torch.no_grad():
            logits = the_hammer(tools)
            
        loss = criterion(logits, is_it_a_hammer)
        
        all_labels.extend(is_it_a_hammer.cpu().detach().numpy().tolist())
        all_preds.extend(logits.argmax(dim=1).detach().cpu().numpy().tolist())
        val_trackers['loss'].update(loss.item())
    
    print('val: {}'.format({k: v.avg for k, v in val_trackers.items()}))
    print(f'val acc: {classification_report(y_true=all_labels, y_pred=all_preds, output_dict=True)["accuracy"]}')
    
    val_loss = val_trackers['loss'].avg
    scheduler.step(val_loss)
    if val_loss < best_loss:
        best_loss = val_loss
        torch.save(the_hammer.state_dict(), 'sledge_best.pth')
        

epoch: 0
train: {'loss': 0.5010050159611114}
train acc: 0.8115784908933218
val: {'loss': 0.47321667808752793}
val acc: 0.7871287128712872
epoch: 1
train: {'loss': 0.3459009795564495}
train acc: 0.8594969644405898
val: {'loss': 0.3605532508630019}
val acc: 0.8452970297029703
epoch: 2
train: {'loss': 0.3148605086623806}
train acc: 0.8594969644405898
val: {'loss': 0.3579517121498401}
val acc: 0.8502475247524752
epoch: 3
train: {'loss': 0.3044641001583779}
train acc: 0.8647007805724197
val: {'loss': 0.3244576293688554}
val acc: 0.8551980198019802
epoch: 4
train: {'loss': 0.2991686024486202}
train acc: 0.8668690372940157
val: {'loss': 0.3221902916064629}
val acc: 0.849009900990099
epoch: 5
train: {'loss': 0.28753265207760953}
train acc: 0.8683868169991327
val: {'loss': 0.3178654003601808}
val acc: 0.8514851485148515
epoch: 6
train: {'loss': 0.290733739733696}
train acc: 0.8688204683434518
val: {'loss': 0.5342272909787985}
val acc: 0.745049504950495
epoch: 7
train: {'loss': 0.281679402475487